In [1]:
import pandas as pd 
import os 
import glob

from create_table import *
from sql_queries import *

In [2]:
cur, conn = connection('127.0.0.1', 'sparkifydb', 'sa', 'P@ssw0rd')

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,"*.json"))
        for f in files:
            all_files.append(os.path.abspath(f))

    return all_files

In [4]:
song_files = get_files("data/song_data")
song_files[0]

'/home/indocyber/course/Data-Modeling-with-SqlServer/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [5]:
df = pd.read_json(song_files[0],lines=True)

df = df.where(pd.notnull(df), None)

df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [6]:
song_data = df[['song_id','title','artist_id','year','duration']].values[0]

print(song_data[0],song_data[1],song_data[2],song_data[3],song_data[4])

SOMZWCG12A8C13C480 I Didn't Mean To ARD7TVE1187B99BFB1 0 218.93179


In [7]:
cur.execute(songs_insert_data,song_data[0],song_data[1],song_data[2],song_data[3],song_data[4])
conn.commit()


In [8]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0]

artist_data

array(['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None],
      dtype=object)

In [9]:
cur.execute(artists_insert_data,artist_data[0],artist_data[1],artist_data[2],artist_data[3],artist_data[4])
conn.commit()

In [10]:
log_files = get_files("data/log_data")
log_files[0]

'/home/indocyber/course/Data-Modeling-with-SqlServer/data/log_data/2018/11/2018-11-01-events.json'

In [11]:
df = pd.read_json(log_files[0],lines=True)
df = df[df.page == "NextSong"]
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [12]:
user_data = df[['userId','firstName','lastName','gender','level']].drop_duplicates().sort_values('userId')

user_data 

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free


In [13]:
for index, row in user_data.iterrows():
    cur.execute(users_insert_data,row.userId, row.firstName, row.lastName, row.gender, row.level)
    conn.commit()

In [14]:
df_time = df.copy()

df_time['ts'] = pd.to_datetime(df_time.ts,unit='ms')

print(df_time.ts, df_time.ts.dt.hour, df_time.ts.dt.day, df_time.ts.dt.dayofweek, df_time.ts.dt.month, df_time.ts.dt.year, df_time.ts.dt.weekday)


2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns] 2     21
4     21
5     21
6     21
7     21
8     21
9     21
10    21
12    21
13    21
14    22
Name: ts, dtype: int64 2     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
12    1
13    1
14    1
Name: ts, dtype: int64 2     3
4     3
5     3
6     3
7     3
8     3
9     3
10    3
12    3
13    3
14    3
Name: ts, dtype: int64 2     11
4     11
5     11
6     11
7     11
8     11
9     11
10    11
12    11
13    11
14    11
Name: ts, dtype: int64 2     2018
4     2018
5     2018
6     2018
7     2018
8     2018
9     2018
10    2018
12    2018
13    2018
14    2018
Name: ts, dtype: int64 2     3
4     3
5     3
6     3
7     3

In [15]:
ts = {
    "starttime" : df_time.ts, 
    "hour" : df_time.ts.dt.hour, 
    "day" : df_time.ts.dt.day, 
    "week" : df_time.ts.dt.dayofweek, 
    "month" : df_time.ts.dt.month, 
    "year" : df_time.ts.dt.year, 
    "weekday" : df_time.ts.dt.weekday
}

df_ts = pd.DataFrame.from_dict(ts)

In [16]:
for index, row in df_ts.iterrows():
    cur.execute(time_insert_data,row.starttime, row.hour, row.day, row.week, row.month, row.year, row.weekday)
    conn.commit()

In [17]:
for index, row in df.iterrows():
    result = cur.execute("select a.song_id, b.artist_id from songs a left join artists b on a.artist_id = b.artist_id where a.title = ? and b.name = ? and b.location = ?", row.song, row.artist, row.location).fetchone()
    songId, artistId = result if result else None, None

    cur.execute(songplays_insert_data,pd.to_datetime(row.ts,unit="ms"), row.userId, row.level, songId, artistId, row.sessionId, row.location, row.userAgent)
    conn.commit()
# df.head()
